In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

iris = load_iris()
features = iris.data
labels = iris.target

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

kernel_types = {
    'linear': SVC(kernel='linear', random_state=42),
    'polynomial': SVC(kernel='poly', degree=3, random_state=42),
    'rbf': SVC(kernel='rbf', random_state=42)
}

results = {}

for name, svm_model in kernel_types.items():
    svm_model.fit(train_features, train_labels)
    predictions = svm_model.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, average='weighted')
    recall = recall_score(test_labels, predictions, average='weighted')
    f1 = f1_score(test_labels, predictions, average='weighted')
    cm = confusion_matrix(test_labels, predictions)
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cm': cm
    }

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d',
                xticklabels=iris.target_names,
                yticklabels=iris.target_names, cmap='Blues')
    plt.title(f'Confusion Matrix - {name.capitalize()} Kernel')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

best_kernel = max(results, key=lambda x: results[x]['f1'])

print(f"Best kernel: {best_kernel.capitalize()}")
print(pd.DataFrame(results).T[['accuracy', 'precision', 'recall', 'f1']])

breast_cancer = load_breast_cancer()
bc_features = breast_cancer.data
bc_labels = breast_cancer.target

bc_train, bc_test, bc_train_labels, bc_test_labels = train_test_split(
    bc_features, bc_labels, test_size=0.2, stratify=bc_labels, random_state=42
)

scaler = StandardScaler()
scaled_train = scaler.fit_transform(bc_train)
scaled_test = scaler.transform(bc_test)

svm_raw = SVC(kernel='rbf', random_state=42)
svm_raw.fit(bc_train, bc_train_labels)
raw_train_pred = svm_raw.predict(bc_train)
raw_test_pred = svm_raw.predict(bc_test)

svm_scaled = SVC(kernel='rbf', random_state=42)
svm_scaled.fit(scaled_train, bc_train_labels)
scaled_train_pred = svm_scaled.predict(scaled_train)
scaled_test_pred = svm_scaled.predict(scaled_test)

print("Without scaling:")
print("Training accuracy:", accuracy_score(bc_train_labels, raw_train_pred))
print("Testing accuracy:", accuracy_score(bc_test_labels, raw_test_pred))
print("With scaling:")
print("Training accuracy:", accuracy_score(bc_train_labels, scaled_train_pred))
print("Testing accuracy:", accuracy_score(bc_test_labels, scaled_test_pred))

print("""
Feature scaling is essential for SVMs—especially those using distance-based kernels like RBF or polynomial.
Scaling ensures all features contribute equally, and improves both accuracy and stability of the algorithm.
""")
